# Ingest gridded products such as ice velocity into OGGM

After running our OGGM experiments we often want to compare the model output to other gridded observations or maybe we want to use additional data sets that are not currently in the [OGGM shop](https://docs.oggm.org/en/stable/input-data.html) to calibrate parameters in the model (e.g. Glen A creep parameter, sliding parameter or the calving constant of proportionality). If you are looking on ways or ideas on how to do this, you are in the right tutorial!

In OGGM, a local map projection is defined for each glacier entity in the RGI inventory following the methods described in [Maussion and others (2019)](https://gmd.copernicus.org/articles/12/909/2019/). The model uses a Transverse Mercator projection centred on the glacier. A lot of data sets, especially those from Polar regions can have a different projections and if we are not careful, we would be making mistakes when we compare them with our model output or when we use such data sets to constrain our model experiments.


**New in OGGM 1.6! We now offer preprocess directories where the data is available already. Visit [OGGM as an accelerator for modelling and machine learning](../10minutes/machine_learning.ipynb) for more info.**


First lets import the modules we need:

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import salem

In [ ]:
from oggm import cfg, utils, workflow, tasks, graphics
cfg.initialize(logging_level='WARNING')

In [ ]:
cfg.PATHS['working_dir'] = utils.gettempdir(dirname='OGGM-shop-on-Flowlines', reset=True)

## Lets define the glaciers for the run 

In [ ]:
rgi_ids = ['RGI60-14.06794']  # Baltoro

In [ ]:
# The RGI version to use
# Size of the map around the glacier.
prepro_border = 80
# Degree of processing level. This is OGGM specific and for the shop 1 is the one you want
from_prepro_level = 3
# URL of the preprocessed gdirs
# we use elevation bands flowlines here
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.3/elev_bands/W5E5'
gdirs = workflow.init_glacier_directories(rgi_ids,
                                          from_prepro_level=from_prepro_level,
                                          prepro_base_url=base_url,
                                          prepro_border=prepro_border)

In [ ]:
gdir = gdirs[0]

In [ ]:
graphics.plot_googlemap(gdir, figsize=(8, 7))

## The `gridded_data` file in the glacier directory

A lot of the data that the model use and produce for a glacier is stored under [the glaciers directories](https://oggm.org/tutorials/stable/notebooks/getting_started.html#glacier-directories) in a NetCDF file called `gridded_data`:

In [ ]:
fpath = gdir.get_filepath('gridded_data')
fpath

In [ ]:
with xr.open_dataset(fpath) as ds:
    ds = ds.load()

In [ ]:
ds

In [ ]:
cmap=salem.get_cmap('topo')
ds.topo.plot(figsize=(7, 4), cmap=cmap);

In [ ]:
ds.glacier_mask.plot();

## Merging the `gridded_data` files of multiple glacier directories

It is also possible to merge the `gridded_data` files of multiple glacier directories. Let's try it with two glaciers:

In [ ]:
rgi_ids_for_merge = ['RGI60-14.06794',  # Baltoro
                     'RGI60-14.07524',  # Siachen
                    ]
gdirs_for_merge = workflow.init_glacier_directories(rgi_ids_for_merge,
                                                    from_prepro_level=from_prepro_level,
                                                    prepro_base_url=base_url,
                                                    prepro_border=prepro_border)

In [ ]:
ds_merged = workflow.merge_gridded_data(
    gdirs_for_merge,
    output_folder=None,  # by default the final file is saved at cfg.PATHS['working_dir']
    output_filename='gridded_data_merged',  # the default file is saved as gridded_data_merged.nc
    included_variables='all',  # you also can provide a list of variables here
    add_topography=False,  # here we can add topography for the new extend
    reset=False,  # set to True if you want to overwrite an already existing file (for playing around)
)

In [ ]:
ds_merged

In [ ]:
ds_merged.glacier_mask.plot()

As you can see, topography is not included because this process requires some time. The reason is that topography is not simply reprojected from the existing gridded_data; instead, it is redownloaded from the source and processed for the new 'merged extent'. You can include topography by setting `add_topography=True` or specifying the DEM source `add_topography='dem_source'`.

There are additional options available, such as `use_glacier_mask` or `preserve_totals` (for preserving the total volume when merging distributed thickness data). For a more comprehensive explanation of these options, refer to the Docstring (by uncommenting the line below).

In [ ]:
# for more available options uncomment the following line
# workflow.merge_gridded_data?

## Add data from OGGM-Shop: bed topography data

Additionally to the data produced by the model, the [OGGM-Shop](https://docs.oggm.org/en/stable/input-data.html) counts with routines that will automatically download and reproject other useful data sets into the glacier projection (For more information also check out this [notebook](https://oggm.org/tutorials/stable/notebooks/oggm_shop.html)). This data will be stored under the file described above. 

OGGM can now download data from the [Farinotti et al., (2019) consensus estimate](https://www.nature.com/articles/s41561-019-0300-3) and reproject it to the glacier directories map:

In [ ]:
from oggm.shop import bedtopo

In [ ]:
workflow.execute_entity_task(bedtopo.add_consensus_thickness, gdirs);

In [ ]:
with xr.open_dataset(gdir.get_filepath('gridded_data')) as ds:
    ds = ds.load()

> the cell below might take a while... be patient

In [ ]:
# plot the salem map background, make countries in grey
smap = ds.salem.get_map(countries=False)
smap.set_shapefile(gdir.read_shapefile('outlines'))
smap.set_topography(ds.topo.data);

In [ ]:
f, ax = plt.subplots(figsize=(9, 9))
smap.set_data(ds.consensus_ice_thickness)
smap.set_cmap('Blues')
smap.plot(ax=ax)
smap.append_colorbar(ax=ax, label='ice thickness (m)');

## OGGM-Shop: velocities

We download data from Millan 2022 (see the [shop](https://docs.oggm.org/en/latest/shop.html#millan-et-al-2022-ice-velocity-and-thickness-products)).

If you want more velocity products, feel free to open a new topic on the OGGM issue tracker!

> this will download severals large datasets **depending on your connection, it might take some time** ...

In [ ]:
# attention downloads data!!!
from oggm.shop import millan22
workflow.execute_entity_task(millan22.velocity_to_gdir, gdirs);

By applying the entity task velocity to gdir OGGM downloads and reprojects the ITS_live files to a given glacier map. 

The velocity components (**vx**, **vy**) are added to the `gridded_data` nc file.

Now we can read in all the gridded data that comes with OGGM, including the velocity components.

In [ ]:
with xr.open_dataset(gdir.get_filepath('gridded_data')) as ds:
    ds = ds.load()
ds

In [ ]:
# plot the salem map background, make countries in grey
smap = ds.salem.get_map(countries=False)
smap.set_shapefile(gdir.read_shapefile('outlines'))
smap.set_topography(ds.topo.data);

In [ ]:
ds

In [ ]:
# get the velocity data
u = ds.millan_vx.where(ds.glacier_mask)
v = ds.millan_vy.where(ds.glacier_mask)
ws = (u**2 + v**2)**0.5

The `.where(ds.glacier_mask)` command will remove the data outside of the glacier outline.

In [ ]:
# get the axes ready
f, ax = plt.subplots(figsize=(12, 12))

# Quiver only every 3rd grid point
us = u[1::5, 1::5]
vs = v[1::5, 1::5]

smap.set_data(ws)
smap.set_cmap('Blues')
smap.plot(ax=ax)
smap.append_colorbar(ax=ax, label = 'ice velocity (m yr$^{-1}$)')

# transform their coordinates to the map reference system and plot the arrows
xx, yy = smap.grid.transform(us.x.values, us.y.values, crs=gdir.grid.proj)
xx, yy = np.meshgrid(xx, yy)
qu = ax.quiver(xx, yy, us.values, vs.values)
qk = ax.quiverkey(qu, 0.82, 0.97, 200, '200 m yr$^{-1}$',
                   labelpos='E', coordinates='axes')

## Bin the data in `gridded_data` into OGGM elevation bands

Now that we have added new data to the `gridded_data` file, we can bin the data sets into the same elevation bands as OGGM by recomputing the elevation bands flowlines:

In [ ]:
tasks.elevation_band_flowline(gdir,
                              bin_variables=['consensus_ice_thickness', 
                                             'millan_vx',
                                             'millan_vy'],
                              preserve_totals=[True, False, False]  # I"m actually not sure if preserving totals is meaningful with velocities - likely not
                              # NOTE: we could bin variables according to max() as well!
                              )

This created a csv in the glacier directory folder with the data binned to it:

In [ ]:
import pandas as pd
df = pd.read_csv(gdir.get_filepath('elevation_band_flowline'), index_col=0)
df

In [ ]:
df['obs_velocity'] = (df['millan_vx']**2 + df['millan_vy']**2)**0.5
df['bed_h'] = df['mean_elevation'] - df['consensus_ice_thickness']

In [ ]:
df[['mean_elevation', 'bed_h', 'obs_velocity']].plot(figsize=(10, 4), secondary_y='obs_velocity');

**The problem with this file is that it does not have a regular spacing**. The numerical model needs regular spacing, which is why OGGM does this:

In [ ]:
# This takes the csv file and prepares new 'inversion_flowlines.pkl' and created a new csv file with regular spacing
tasks.fixed_dx_elevation_band_flowline(gdir,
                                       bin_variables=['consensus_ice_thickness',
                                                      'millan_vx', 
                                                      'millan_vy'],
                                       preserve_totals=[True, False, False]
                                      )

In [ ]:
df_regular = pd.read_csv(gdir.get_filepath('elevation_band_flowline', filesuffix='_fixed_dx'), index_col=0)
df_regular

The other variables have disappeared for saving space, but I think it would be nicer to have them here as well. We can grab them from the inversion flowlines (this could be better handled in OGGM):

In [ ]:
fl = gdir.read_pickle('inversion_flowlines')[0]
df_regular['mean_elevation'] = fl.surface_h

In [ ]:
df_regular['obs_velocity'] = (df_regular['millan_vx']**2 + df_regular['millan_vy']**2)**0.5
df_regular['consensus_bed_h'] = df_regular['mean_elevation'] - df_regular['consensus_ice_thickness']

In [ ]:
df_regular[['mean_elevation', 'consensus_bed_h', 'obs_velocity']].plot(figsize=(10, 4),
                                                                       secondary_y='obs_velocity');

OK so more or less the same but this time on a regular grid. Note that these now have the same length as the OGGM inversion flowlines, i.e. one can do stuff such as comparing what OGGM has done for the inversion:

In [ ]:
inv = gdir.read_pickle('inversion_output')[0]

In [ ]:
df_regular['OGGM_bed'] = df_regular['mean_elevation'] - inv['thick']

In [ ]:
df_regular[['mean_elevation', 'consensus_bed_h', 'OGGM_bed']].plot();

## Compare velocities: at inversion

OGGM already inverted the ice thickness based on some optimisation such as matching the regional totals. Which parameters did we use?

In [ ]:
d = gdir.get_diagnostics()
d

In [ ]:
# OK set them so that we are consistent
cfg.PARAMS['inversion_glen_a'] = d['inversion_glen_a']
cfg.PARAMS['inversion_fs'] = d['inversion_fs']

In [ ]:
# Since we overwrote the inversion flowlines we have to do stuff again
tasks.mb_calibration_from_geodetic_mb(gdir,
                                      informed_threestep=True,
                                      overwrite_gdir=True)
tasks.apparent_mb_from_any_mb(gdir)
tasks.prepare_for_inversion(gdir)
tasks.mass_conservation_inversion(gdir)
tasks.compute_inversion_velocities(gdir)

In [ ]:
inv = gdir.read_pickle('inversion_output')[0]

In [ ]:
df_regular['OGGM_inversion_velocity'] = inv['u_surface']

In [ ]:
df_regular[['obs_velocity', 'OGGM_inversion_velocity']].plot();

The velocities are higher, because: 
- OGGM is at equilibrium
- the comparison between the bulk velocities of OGGM and the gridded observed ones is not straightforward.

## Velocities during the run 

TODO: here we could use velocities from the spinup run!!

Inversion velocities are for a glacier at equilibrium - this is not always meaningful. Lets do a run and store the velocities with time:

In [ ]:
cfg.PARAMS['store_fl_diagnostics'] = True

In [ ]:
tasks.run_from_climate_data(gdir);

In [ ]:
with xr.open_dataset(gdir.get_filepath('model_diagnostics')) as ds_diag:
    ds_diag = ds_diag.load()

In [ ]:
ds_diag.volume_m3.plot();

In [ ]:
with xr.open_dataset(gdir.get_filepath('fl_diagnostics'), group='fl_0') as ds_fl:
    ds_fl = ds_fl.load()

In [ ]:
ds_fl

In [ ]:
ds_fl.sel(time=[2003, 2010, 2020]).ice_velocity_myr.plot(hue='time');

In [ ]:
# The OGGM model flowlines also have the downstream lines
df_regular['OGGM_velocity_run_begin'] = ds_fl.sel(time=2003).ice_velocity_myr.data[:len(df_regular)]
df_regular['OGGM_velocity_run_end'] = ds_fl.sel(time=2020).ice_velocity_myr.data[:len(df_regular)]

In [ ]:
df_regular[['obs_velocity', 'OGGM_inversion_velocity',
            'OGGM_velocity_run_begin', 'OGGM_velocity_run_end']].plot();

## What's next?

- return to the [OGGM documentation](https://docs.oggm.org)
- back to the [table of contents](../welcome.ipynb)